In [94]:
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET

# get list of file names
files = [os.path.join('tota-master', 'xml', path) for path in os.listdir(os.path.join('tota-master', 'xml'))]

columns = ['Name', 'Type', 'WTO ID', 'Identifier', 'Date signed', 'Date into force', 'Date of notification', 'End of implementation', 'Date inactive', 'Original parties', 'Current parties', 'Composition', 'Region', 'All WTO', 'Cross Regional', 'Language', 'Text', 'Articles']
df = pd.DataFrame(columns=columns)

# populate data frame
for (idx, file) in enumerate(files):
    root = ET.parse(file).getroot()
    meta = root[0]
    body = root[1]
    
    name = meta.find('name').text
    _type = meta.find('type').text
    _id = meta.find('wto_rta_id').text
    identifier = meta.find('treaty_identifier').text
    date_signed = meta.find('date_signed').text
    date_into_force = meta.find('date_into_force').text
    date_notification = meta.find('date_notification').text
    end_implementation = meta.find('end_implementation').text
    date_inactive = meta.find('date_inactive').text
    original_parties = [party.text for party in meta.findall('./parties_original/partyisocode')]
    current_parties = [party.text for party in meta.findall('./parties/partyisocode')]
    composition = meta.find('composition').text
    region = meta.find('region').text
    all_wto = meta.find('parties_wto').text
    cross_regional = meta.find('crossregional').text
    language = meta.find('language').text
    text = ''.join([article.text for article in body.findall('.//chapter/article')])
#     chapter_names = [chapter.attrib['name'] if 'name' in chapter.attrib else 'Untitled' for chapter in body.findall('chapter')]
#     chapter_article_count =  [len(chapter.findall('article')) for chapter in body.findall('chapter')]
#     chapter_id = [chapter.attrib['chapter_identifier'] for chapter in body.findall('chapter')]
    articles = [article.text for article in body.findall('.//chapter/article')]
#     article_names = [article.attrib['name'] if 'name' in article.attrib else 'Untitled' for article in body.findall('.//chapter/article')]
#     article_id = [article.attrib['article_identifier'] for article in body.findall('.//chapter/article')]
    df.loc[idx] = [name, _type, _id, identifier, date_signed, date_into_force, date_notification, end_implementation, date_inactive, original_parties, current_parties, composition, region, all_wto, cross_regional, language, text, articles]
    
# sort data frame chronologically
df.sort_values(by='Date signed', inplace=True)

df.loc[1,'Articles']


['THE EUROPEAN ECONOMIC COMMUNITY, \nof the one part, and \n\n\nTHE PORTUGUESE REPUBLIC, \nof the other part, \n\n\nDESIRING to consolidate and to extend, upon the enlargement of \nthe European Economic Community, the economic relations existing between the Community and Portugal and to ensure, with due \nregard for fair conditions of competition, the harmonious \ndevelopment of their commerce for the purpose of contributing \nto the work of constructing Europe, \n\n\nRESOLVED to this end to eliminate progressively the obstacles \nto substantially all their trade, in accordance with the provisions of the General Agreement on Tariffs and Trade \nconcerning the establishment of free trade areas, \n\n\nDECLARING their readiness to examine, in the light of any \nrelevant factor, and in particular of developments in the \nCommunity, the possibility of developing and deepening their \nrelations where it would appear to be useful in the interests \nof their economies to extend them to fields 

In [95]:
# only look at English, bilateral trade agreements
df_new = df.loc[(df["Language"] == "en") & (df["Composition"] == "Bilateral")]

In [96]:
# set the index to the identifier as on GitHub
df_new = df_new.set_index("Identifier")
df_new.index = df_new.index.astype('int')

In [97]:
# delete columns with missing data
df_new = df_new.dropna(axis=1)

In [98]:
# search for anomalies
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_new)

                                                         Name  \
Identifier                                                      
310            South Africa - Southern Rhodesia Customs Union   
254                   El Salvador - Nicaragua Free Trade Area   
266                       Ghana - Upper Volta Trade Agreement   
174              Australia - New Zealand Free Trade Agreement   
274                  Ireland - United Kingdom Free Trade Area   
126                     Australia - Papua New Guinea (PATCRA)   
122                        Australia - New Zealand (ANZCERTA)   
121                                               US - Israel   
186                 Canada - US Free Trade Agreement (CUSFTA)   
120               Lao People's Democratic Republic - Thailand   
262                                Finland - Estonia Protocol   
277                      Latvia - Sweden Free Trade Agreement   
280                   Lithuania - Sweden Free Trade Agreement   
259                     E

In [99]:
# drop these agreements because they include the former country of Serbia and Montenegro which was split
df_new = df_new.drop([286, 295])

In [100]:
# set new columns for party 1 and party 2
df_new["Party 1"] = df_new["Original parties"].str[0]
df_new["Party 2"] = df_new["Original parties"].str[1]

In [101]:
df_new = df_new.drop("Original parties", axis=1)
df_new = df_new.drop("Current parties", axis=1)

In [102]:
df_new['Total articles'] = df_new['Articles'].str.len()

In [103]:
df_new['Total words'] = df_new['Articles'].apply(lambda x: ','.join(map(str, x))).str.len()

In [104]:
df_new['Number of unique words'] = df_new['Articles'].apply(lambda x: ','.join(map(str, x))).str.split(' ', expand=True).apply(lambda x: list(set(x)), axis=1).str.len()


In [105]:
df_new.to_csv("fta_clean.csv")

In [106]:
df_new

,Name,Type,WTO ID,Date signed,Date into force,Composition,Region,All WTO,Cross Regional,Language,Text,Articles,Party 1,Party 2,Total articles,Total words,Number of unique words
Identifier,,,,,,,,,,,,,,,,,
310,South Africa - Southern Rhodesia Customs Union,Customs Union,360,1948-12-06,1949-04-01,Bilateral,Africa,Yes,No,en,""".... to state, for the informationof the CONT...","["".... to state, for the informationof the CON...",ZAF,ZWE,1,4852,362
254,El Salvador - Nicaragua Free Trade Area,Free Trade Agreement,297,1951-03-09,1951-08-21,Bilateral,Central America,Yes,No,en,FREE-TRADE TREATY BETWEEN THE REPUBLICS OF NIC...,[FREE-TRADE TREATY BETWEEN THE REPUBLICS OF NI...,NIC,SLV,26,16892,913
266,Ghana - Upper Volta Trade Agreement,Free Trade Agreement,315,1961-06-28,1962-05-09,Bilateral,Africa,Yes,No,en,RESTRICTED \n\n\nGENERAL AGREEMENT ON L/1766 1...,[RESTRICTED \n\n\nGENERAL AGREEMENT ON L/1766 ...,BFA,GHA,18,8209,509
174,Australia - New Zealand Free Trade Agreement,Free Trade Agreement,200,1965-08-31,1966-01-01,Bilateral,Oceania,Yes,No,en,The Government of New Zealand and the Governme...,[The Government of New Zealand and the Governm...,AUS,NZL,19,27108,1049
274,Ireland - United Kingdom Free Trade Area,Free Trade Agreement,323,1965-12-14,1966-07-01,Bilateral,Europe,Yes,No,en,UNITED KINGDOM/IRELAND \nFREE-TRADE AREA AGREE...,[UNITED KINGDOM/IRELAND \nFREE-TRADE AREA AGRE...,GBR,IRL,29,46193,1760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,Rep. of Korea - New Zealand,Free Trade Agreement & Economic Integration Ag...,643,2015-03-23,2015-12-20,Bilateral,East Asia; Oceania,Yes,Yes,en,FREE TRADE AGREEMENT BETWEEN \nNEW ZEALAND AND...,[FREE TRADE AGREEMENT BETWEEN \nNEW ZEALAND AN...,KOR,NZL,254,338558,6268
401,Rep. of Korea - Viet Nam,Free Trade Agreement & Economic Integration Ag...,709,2015-05-05,2015-12-20,Bilateral,East Asia,Yes,No,en,The Government of the Republic of Korea (“Kore...,[The Government of the Republic of Korea (“Kor...,KOR,VNM,199,247035,5271
398,China - Rep. of Korea,Free Trade Agreement & Economic Integration Ag...,697,2015-06-01,2015-12-20,Bilateral,East Asia,Yes,No,en,FREE TRADE AGREEMENT BETWEEN \nTHE GOVERNMENT ...,[FREE TRADE AGREEMENT BETWEEN \nTHE GOVERNMENT...,CHN,KOR,307,318986,6309


In [133]:
articles = df_new[['Articles']].explode('Articles')
articles.to_csv('articles.csv')

In [134]:
df_new.loc[171,:]

Name                                                 Albania - UNMIC/Kosovo
Type                                                   Free Trade Agreement
WTO ID                                                                  197
Date signed                                                      2003-07-07
Date into force                                                  2003-10-01
Composition                                                       Bilateral
Region                                                               Europe
All WTO                                                                  No
Cross Regional                                                           No
Language                                                                 en
Text                      1. The Contracting Parties shall gradually est...
Articles                  [1. The Contracting Parties shall gradually es...
Party 1                                                                 ALB
Party 2     